# ComfyUI for InfiniteTalk (Wan2.1) on Colab

This notebook sets up a ComfyUI environment optimized for **Tesla T4 (16GB)** usage with **InfiniteTalk**.

It includes:
- **ComfyUI-WanVideoWrapper** (Kijai)
- **InfiniteTalk Custom Nodes** (MeiGen-AI)
- **FP8 Model Support** (Critical for 16GB VRAM)
- **Cloudflared Tunnel** for public access

In [ ]:
import os
from pathlib import Path

# 1. Install ComfyUI
if not os.path.exists("ComfyUI"):
    !git clone https://github.com/comfyanonymous/ComfyUI
    %cd ComfyUI
    !pip install -r requirements.txt
else:
    %cd ComfyUI

# 2. Install Manager & Custom Nodes
!git clone https://github.com/ltdrdata/ComfyUI-Manager custom_nodes/ComfyUI-Manager
!git clone https://github.com/kijai/ComfyUI-WanVideoWrapper custom_nodes/ComfyUI-WanVideoWrapper
!git clone -b comfyui https://github.com/MeiGen-AI/InfiniteTalk custom_nodes/ComfyUI-InfiniteTalk

# Install dependencies for custom nodes
!pip install -r custom_nodes/ComfyUI-WanVideoWrapper/requirements.txt
!pip install -r custom_nodes/ComfyUI-InfiniteTalk/requirements.txt

# 3. Flash Attention (Fix for PyTorch 2.9/Python 3.12)
!pip install --no-cache-dir packaging ninja psutil wheel
try:
    import flash_attn
except ImportError:
    print("Installing Optimized Flash Attention...")
    !pip install --no-cache-dir https://github.com/Dao-AILab/flash-attention/releases/download/v2.8.3/flash_attn-2.8.3+cu12torch2.9cxx11abiTRUE-cp312-cp312-linux_x86_64.whl

In [ ]:
# 4. Download Models (FP8 for T4 Compatibility)
import os

# Define paths
models_dir = "models"
diffusion_dir = f"{models_dir}/diffusion_models"
text_enc_dir = f"{models_dir}/text_encoders"
vae_dir = f"{models_dir}/vae"
clip_dir = f"{models_dir}/clip_vision"
checkpoint_dir = f"{models_dir}/checkpoints"

# Ensure directories exist
os.makedirs(diffusion_dir, exist_ok=True)
os.makedirs(text_enc_dir, exist_ok=True)
os.makedirs(vae_dir, exist_ok=True)
os.makedirs(clip_dir, exist_ok=True)
os.makedirs(checkpoint_dir, exist_ok=True)

print("Downloading Optimized FP8 Models...")

# Wan2.1 Diffusion Model (FP8 - 14B)
if not os.path.exists(f"{diffusion_dir}/wan2.1_i2v_480p_14B_fp8_e4m3fn.safetensors"):
    !wget -c https://huggingface.co/Comfy-Org/Wan_2.1_ComfyUI_repackaged/resolve/main/split_files/diffusion_models/wan2.1_i2v_480p_14B_fp8_e4m3fn.safetensors -P {diffusion_dir}

# Text Encoder (UMT5 FP8)
if not os.path.exists(f"{text_enc_dir}/umt5_xxl_fp8_e4m3fn_scaled.safetensors"):
    !wget -c https://huggingface.co/Comfy-Org/Wan_2.1_ComfyUI_repackaged/resolve/main/split_files/text_encoders/umt5_xxl_fp8_e4m3fn_scaled.safetensors -P {text_enc_dir}

# VAE
if not os.path.exists(f"{vae_dir}/wan_2.1_vae.safetensors"):
    !wget -c https://huggingface.co/Comfy-Org/Wan_2.1_ComfyUI_repackaged/resolve/main/split_files/vae/wan_2.1_vae.safetensors -P {vae_dir}

# CLIP Vision
if not os.path.exists(f"{clip_dir}/clip_vision_h.safetensors"):
    !wget -c https://huggingface.co/Comfy-Org/Wan_2.1_ComfyUI_repackaged/resolve/main/split_files/clip_vision/clip_vision_h.safetensors -P {clip_dir}

# InfiniteTalk Checkpoints (MeiGen)
if not os.path.exists(f"{checkpoint_dir}/infinitetalk_multi_fp8.safetensors"):
    !wget -c https://huggingface.co/MeiGen-AI/InfiniteTalk/resolve/main/quant_models/infinitetalk_multi_fp8.safetensors -P {checkpoint_dir}

print("✅ All Models Downloaded!")

In [ ]:
# 5. Install Cloudflared & Launch
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb

print("🚀 Starting ComfyUI...")
print("Look for the trycloudflare.com URL below to access the interface.\t")

# Run ComfyUI in background, expose via tunnel
!python main.py --listen --port 8188 & cloudflared tunnel --url http://localhost:8188